In [2]:
!pip install servicex_analysis_utils

  Using cached servicex_analysis_utils-1.2.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached servicex-3.2.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached aioboto3-15.5.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached func_adl-3.5.0-py3-none-any.whl.metadata (12 kB)
  Using cached httpx_retries-0.4.5-py3-none-any.whl.metadata (4.1 kB)
  Using cached make_it_sync-2.1.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached qastle-0.19.0-py3-none-any.whl.metadata (8.1 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached tinydb-4.8.2-py3-none-any.whl.metadata (6.7 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached types_pyyaml-6.0.12.20250915-py3-none-any.whl.metadata (1.7 kB)
  Using cached aiobotocore-2.25.1-py3-none-any.whl.metadata (25 kB)
  Using cached aiofiles-25.1.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached botocore-1.40.61-py3-none-any.whl.meta

In [6]:
from pathlib import Path
import sys

sys.path.append(str(Path().resolve().parent))

from servicex import deliver, query 
from servicex_analysis_utils import ds_type_resolver, to_awk
import utils
import gzip
import json

In [8]:
fname = "../ntuple_production/file_metadata.json.gz"
with gzip.open(fname) as f:
    dataset_info = json.loads(f.read().decode())

In [9]:
total_sig=0
signal_containers={}
for container, metadata in dataset_info["Hplus_cb"].items():
    evts = metadata["nevts_input"]
    total_sig+=evts
    mass = utils.hplus_signal_mass(container)
    _ , _, campaign = utils.dsid_rtag_campaign(container)
    signal_containers[f"{mass}_{campaign}"]={'DSID' : container, 'Events': evts }
print(f"H+ -> cb signal in {len(signal_containers)} DSIDs for {total_sig:.0e} total events")

H+ -> cb signal in 40 DSIDs for 9e+06 total events


In [10]:
signal_containers['20GeV_mc20e']['Events']

320000

In [11]:
sample = ds_type_resolver(signal_containers['20GeV_mc20e']['DSID'])
sample

In [16]:
config_path = "../ntuple_production/reco.yaml" 

In [17]:
request = query.TopCP(reco=config_path, max_events = 200_000)
request

TopCPQuery(reco='../ntuple_production/reco.yaml', parton=None, particle=None, max_events=200000, no_systematics=True, no_filter=False)

In [18]:
spec = {
        "General": {"OutputDirectory": "."},
        "Sample": [{
            "Name": '20GeV_mc20e',
            "Dataset": sample,
            "Query": request,     
        }]
    }

In [19]:
files = deliver(spec)

Output()